In [109]:
from sklearn.datasets import load_iris
iris = load_iris()
X,y = iris.data, iris.target

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1, test_size = 0.1)

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

y_train_onehot = keras.utils.to_categorical(y_train)

In [165]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from sklearn.model_selection import cross_val_score

def model_keras ():
    tf.set_random_seed(1)
    model = keras.models.Sequential()
    
    model.add(Dense(units = 100, input_dim = 4,activation = "relu"))
    model.add(Dense(units = 100, activation = "relu"))
    model.add(Dense(units = y_train_onehot.shape[1], activation = "softmax"))

    model.compile(optimizer = Adam(lr = 0.001), metrics = ["accuracy"], loss = "categorical_crossentropy")
    #model.fit(X_train_std,y_train_onehot,batch_size=10,epochs = 20,verbose = False)

    return model

from keras.wrappers.scikit_learn import KerasClassifier
model_wrapper_scikit_learn = KerasClassifier(build_fn = model_keras,
                                            epochs = 10,
                                            batch_size = 10,
                                            verbose = True)

cross_val_score(model_wrapper_scikit_learn,X = X_train,y = y_train,cv = 3,verbose = True)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
90/90 [==============================] - 3s 34ms/step - loss: 0.9378 - acc: 0.5889
Epoch 2/10
90/90 [==============================] - 0s 389us/step - loss: 0.7543 - acc: 0.7556
Epoch 3/10
90/90 [==============================] - 0s 454us/step - loss: 0.6576 - acc: 0.7667
Epoch 4/10
90/90 [==============================] - 0s 517us/step - loss: 0.5647 - acc: 0.7778
Epoch 5/10
90/90 [==============================] - 0s 518us/step - loss: 0.4820 - acc: 0.9111
Epoch 6/10
90/90 [==============================] - 0s 505us/step - loss: 0.4379 - acc: 0.8556
Epoch 7/10
90/90 [==============================] - 0s 470us/step - loss: 0.3948 - acc: 0.9333
Epoch 8/10
90/90 [==============================] - 0s 495us/step - loss: 0.3686 - acc: 0.9111
Epoch 9/10
90/90 [==============================] - 0s 513us/step - loss: 0.3307 - acc: 0.9444
Epoch 10/10
45/45 [==============================] - 1s 32ms/step
Epoch 1/10
90/90 [==============================] - 4s 44ms/step - loss: 1.0532 

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   19.9s finished


array([0.82222221, 0.97777777, 0.9111111 ])

In [147]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, random_state=1).split(X_train_std, y_train)
model = model_keras()
for k, (train,test) in enumerate(kfold):
    
    model.fit(X_train_std[train],keras.utils.to_categorical(y_train[train]),verbose = False)
    
    y_pred_onehot = model.predict(X_train_std[test])
    
    accuracy = accuracy_score(y_train[test],y_pred_onehot.argmax(axis = 1))
    print(accuracy)

0.9333333333333333
1.0
1.0
1.0
0.9285714285714286
1.0
1.0
1.0
1.0
0.9166666666666666
